In [ ]:
# Configurable colab notebook for VITS contributed by 蓝琪猫（https://space.bilibili.com/455034987）

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)


# 新段落

安装依赖


In [ ]:
%cd /content/drive/MyDrive/vits-mandarin-windows-mandarin
!pip install -r requirements.txt
!pip install demjson
!pip install noisereduce
!pip install unidecode
!pip install phonemizer

对齐

In [ ]:
%cd /content/drive/MyDrive/vits-mandarin-windows-mandarin/monotonic_align
!python setup.py build_ext --inplace
%cd ..

In [ ]:
%cd root/content/drive/MyDrive/vits-mandarin-windows-mandarin
!python preprocess.py --text_index 1 --filelists filelists/wz_train.txt
!python preprocess.py --text_index 1 --filelists filelists/wz_val.txt

In [ ]:
#@title 生成配置文件
# forked from https://github.com/rotten-work/vits-mandarin-windows/blob/mandarin/configs/bb_base.json
#@markdown 训练次数
set_epochs = 15001 #@param {type:"integer"}
#@markdown 训练集位置
set_training_files = "filelists/xia_train.txt" #@param {type:"string"}
#@markdown 验证集位置
set_validation_files = "filelists/xia_val.txt"#@param {type:"string"}
#@markdown 每隔多少次step保存一次
set_eval_interval = 1000 #@param {type:"integer"}
#@markdown 单次step的文件数
set_batch_size = 12 #@param {type:"integer"}
#@markdown 配置文件名称
set_json_name = "wz2mx.json" #@param {type:"string"}

training_json = {
  "train": {
    "log_interval": 200,
    "eval_interval": set_eval_interval,
    "seed": 1234 ,
    "epochs": set_epochs,
    "learning_rate": 2e-4,
    "betas": [0.8, 0.99],
    "eps": 1e-9,
    "batch_size": set_batch_size,
    "fp16_run": True,
    "lr_decay": 0.999875,
    "segment_size": 8192,
    "init_lr_ratio": 1,
    "warmup_epochs": 0,
    "c_mel": 45,
    "c_kl": 1.0
  },
  "data": {
    "training_files": set_training_files + ".cleaned",
    "validation_files": set_validation_files + ".cleaned",
    "language": "cmn",
    "text_cleaners":["chinese_cleaners"],
    "max_wav_value": 32768.0,
    "sampling_rate": 22050,
    "filter_length": 1024,
    "hop_length": 256,
    "win_length": 1024,
    "n_mel_channels": 80,
    "mel_fmin": 0.0,
    "mel_fmax": None,
    "add_blank": True,
    "n_speakers": 0,
    "cleaned_text": True
  },
  "model": {
    "inter_channels": 192,
    "hidden_channels": 192,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5], [1,3,5], [1,3,5]],
    "upsample_rates": [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,4,4],
    "n_layers_q": 3,
    "use_spectral_norm": False
  },
  "data_loader": {
    "use_train_sampler": True,
    "num_workers": 2
  },
  "checkpoints": {
    "auto_delete_old_checkpoints": True,
    "replace_old_checkpoints_mode": False,
    "num_checkpoints_to_keep": 4
  }
}

import demjson
import os
os.chdir('/content/drive/MyDrive/vits-mandarin-windows-mandarin/configs')
training_json_text = demjson.encode(training_json)
with open(set_json_name, "w") as file:
  file.write(training_json_text)
  os.chdir('/content/drive/MyDrive/vits-mandarin-windows-mandarin')

开始训练

In [ ]:
os.environ['PYTHONPATH']='.'
!CUDA_VISIBLE_DEVICES=0

In [ ]:
%cd /content/drive/MyDrive/vits-mandarin-windows-mandarin
!python train.py -c configs/xiamx.json -m xia

推理（暂时）

In [ ]:
%cd /content/drive/MyDrive/vits-mandarin-windows-mandarin
!python inference.py